# Import Library

In [1]:
from datetime import datetime, timedelta
import os
import pandas as pd
import requests
import json
import re

# Azure
import openai
from openai import AzureOpenAI

# Integrasi dengan PostgreSQL
import psycopg2
from psycopg2 import pool 
import ast
import pgvector
import math
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector

# Connect to PostreSQL

In [2]:
host = "c-skillup-vector.hh3bwvk75ljfl6.postgres.cosmos.azure.com"
dbname = "skillup-citus"
user = "citus"
password = "SkillUp123"
sslmode = "require"

conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

postgreSQL_pool = None
if (not postgreSQL_pool):
    print("Connection pool created successfully")
    postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20,conn_string)
    
    conn = postgreSQL_pool.getconn()

cursor = conn.cursor()

Connection pool created successfully


# Embedding Connect

In [3]:
url = "https://skillupllm.openai.azure.com/openai/deployments/SkillUp-Embedding/embeddings?api-version=2024-02-01"

headers = {
  'Content-Type': 'application/json',
  'api-key': '8575d0680b2e48599a9058bf62713335'
}

# Create Table

## Table Hasil Scrap

In [4]:
try :
    # Create table to store embeddings and metadata
    table_create_command = """
    CREATE TABLE data_scrapping (
        id bigserial primary key, 
        company_names text,
        job_titles text,
        posted_on date,
        job_details text,
        skill text,
        images text,
        type_location text,
        location text,
        embedding vector(1536)
    );
    """

    cursor.execute(table_create_command)
    # cursor.close()
    conn.commit()
except Exception as e :
    print("Table Exists")

# Store Data to Database

In [ ]:
host = "c-skillup-vector.hh3bwvk75ljfl6.postgres.cosmos.azure.com"
dbname = "skillup-citus"
user = "citus"
password = "SkillUp123"
sslmode = "require"

conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)

postgreSQL_pool = None
if (not postgreSQL_pool):
    print("Connection pool created successfully")
    postgreSQL_pool = psycopg2.pool.SimpleConnectionPool(1, 20,conn_string)
    
    conn = postgreSQL_pool.getconn()

cursor = conn.cursor()

def select_data(cursor, conn):
    # Tentukan path folder
    folder_path = 'D:\Codelabs\Kompetisi\AIC Compfest\Projek'

    # Ambil semua nama file dengan ekstensi .csv di dalam folder
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    for csv in csv_files:
        print("Nama File : ",csv)
        df_data = pd.read_csv(str(csv))
        df_data.fillna("")
        print("Jumlah Data : ",df_data.shape)
        # Terapkan fungsi ke kolom 'Posted On'
        df_data['Posted On'] = df_data['Posted On'].apply(convert_posted_on)
        df_data['all_text'] = df_data['Job Titles'] + ' ' + df_data['Job Details'] + ' ' + df_data['Skill']
        df_data['all_text'] = df_data['all_text'].str.replace('\n', '', regex=False)
        df_data['Text Embed'] = df_data['all_text'].apply(embed_text)
        df_data.dropna(subset="Text Embed", inplace=True)
        display(df_data['Text Embed'])
        store_data(cursor, conn, df_data)

def embed_text(title) :
    try:
        payload = json.dumps({"input": title})
        print("Ini payload : ",payload)
        # Send a POST request and get the query embedding
        responses = requests.request("POST", url, headers=headers, data=payload)
        print(responses.json()['data'][0]['embedding'])
        query_embedding = responses.json()['data'][0]['embedding']
        return query_embedding
    except Exception as e:
        return None

def convert_posted_on(posted_on):
    # Tangani kasus yang ada
    if 'Reposted' in posted_on:
        # Ambil minggu dari "Reposted" jika ada
        weeks = int(posted_on.split()[1])
        return datetime.now() - timedelta(weeks=weeks)
    elif 'week ago' in posted_on:
        weeks = int(posted_on.split()[0])
        return datetime.now() - timedelta(weeks=weeks)
    elif 'weeks ago' in posted_on:
        weeks = int(posted_on.split()[0])
        return datetime.now() - timedelta(weeks=weeks)
    elif 'month ago' in posted_on:
        months = int(posted_on.split()[0])
        return datetime.now() - timedelta(weeks=months*4)  # Estimasi 1 bulan = 4 minggu
    elif 'day ago' in posted_on:
        days = int(posted_on.split()[0])
        return datetime.now() - timedelta(days=days)
    elif 'hours ago' in posted_on:
        hours = int(posted_on.split()[0])
        return datetime.now() - timedelta(hours=hours)
    else:
        return datetime.now()  # Jika tidak ada format yang dikenali, kembalikan tanggal sekarang

def store_data(cursor, conn, df_data):
    data_list = [
        (
            row['Company Names'], 
            row['Job Titles'], 
            row['Posted On'], 
            row['Job Details'], 
            row['Skill'], 
            row['Images'], 
            row['Type Location'], 
            row['Locations'],
            row['Text Embed']
        ) 
        for index, row in df_data.iterrows()
    ]

    execute_values(cursor, 
        "INSERT INTO data_scrapping (company_names, job_titles, posted_on, job_details, skill, images, type_location, location, embedding) VALUES %s", 
        data_list
    )

    # Commit setelah melakukan insert
    conn.commit()

select_data(cursor, conn)

# Get Relevant Data

In [90]:
job_category = "Website"
payload = json.dumps({"input": job_category})
# Send a POST request and get the query embedding
responses = requests.request("POST", url, headers=headers, data=payload)
query_embedding = responses.json()['data'][0]['embedding']

In [91]:
try:
    # Execute the SQL query to retrieve relevant data
    cursor.execute("""
                    SELECT company_names, job_titles, posted_on, job_details, skill, images, type_location, location,
                    (1 - (embedding <=> %s)) as similarity FROM data_scrapping 
                    WHERE 1 - (embedding <=> %s) > %s
                    ORDER BY similarity DESC
                    """, (str(query_embedding), str(query_embedding), str(0.1)))
except Exception as e:
    print("error : ", e)

# Fetch all the data
data = cursor.fetchall()

# Get the column names from the cursor description
column_names = [desc[0] for desc in cursor.description]

# Convert the data to a pandas DataFrame
df = pd.DataFrame(data, columns=column_names)

In [85]:
df.to_csv("tes.csv", index=False)

# Get Tren Jobs

## Cleaning

In [92]:
# Fungsi untuk membersihkan job_titles
def clean_job_title(title):
    title = str(title).lower()
    if "science" in title.lower():
        title = title.lower().replace("science", "scientist")
    if "engineering" in title.lower():
        title = title.lower().replace("engineering", "engineer")
    if "analytics" in title.lower():
        title = title.lower().replace("analytics", "analyst")
    if "ds" in title.lower():
        title = title.lower().replace("ds", "data scientist")
    if "front end" in title.lower():
        title = title.lower().replace("front end", "frontend")
    if "(" in title.lower():
        title = title.lower().replace("(", ",")
    if "lead" in title.lower():
        title = title.lower().replace("lead", "")
    if "senior" in title.lower():
        title = title.lower().replace("senior", "")
    if "remote" in title.lower():
        title = title.lower().replace("remote", "")
    if "sr." in title.lower():
        title = title.lower().replace("sr.", "")
    # if "data" in title.lower():
    #     title = title.lower().replace("data", "")
    # Menghapus kata 'intern', 'internship', dan 'freelance'
    title = re.sub(r'\b(intern(ship)?|freelance)\b', '', title, flags=re.IGNORECASE)
    title = re.sub(r'[()]', '', title)
            
    return title.strip()

# Menerapkan fungsi ke kolom 'Job Title'
df.drop_duplicates(inplace=True)
df.drop_duplicates(subset=['company_names','job_titles','skill','type_location','location'])
df['job_titles'] = df['job_titles'].apply(clean_job_title)

## Azure

In [93]:
client = AzureOpenAI(
    api_key="8575d0680b2e48599a9058bf62713335",  
    api_version="2024-02-01",
    azure_endpoint="https://skillupllm.openai.azure.com/"
)
    
deployment_name = 'SkillUp-4o'

In [94]:
prompt_final = '''
ini adalah list job title Backend dari hasil pengambilan data di linkedin. berikan saya 5 job yang paling banyak dicari
'''+str(df['job_titles'].to_list())+'''

Terdapat aturan yang harus kamu patuhi :
1. Hasil keluaran wajib dalam format seperti contoh di bawah :
[
 {'job_title':'Nama Job','total':45},
 {'job_title':'Nama Job','total':17},
 {'job_title':'Nama Job','total':11},
 {'job_title':'Nama Job','total':10},
 {'job_title':'Nama Job','total':6}
]
2. Tidak boleh ada keluaran lain selain pada poin 1
3. Jangan sampai ada nama tempat di hasilnya, betul betul hanya nama job
4. Jangan sampe ada yang double job misal 'data warehouse data engineer'
5. Pekerjaan yang dicari adalah yang berkaitan dengan '''+job_category+'''
6. Jangan sampai ada nama job yang double, misal sebelumnya sudah ada "data engineer" namun di nama job berikutnya terdapat "it data engineer" cukup satu saja "data engineer""'''


In [95]:
response = client.chat.completions.create(model=deployment_name, messages=[{"role": "user", "content": prompt_final}], stream=False, max_tokens=2000, temperature=0 )
result = response.choices[0].message.content

# Membersihkan string dan mengganti tanda kutip tunggal dengan kutip ganda
cleaned_string = result.replace("'", '"').replace("\n", "").replace("```json", "").replace("```", "").strip()

# Mengonversi string menjadi JSON
json_data = json.loads(cleaned_string)

# Menampilkan hasil
print(json_data)

[{'job_title': 'web developer', 'total': 7}, {'job_title': 'fullstack web developer', 'total': 5}, {'job_title': 'backend developer', 'total': 5}, {'job_title': 'software engineer', 'total': 5}, {'job_title': 'frontend developer', 'total': 4}]
